In [1]:
import pandas as pd
import numpy as np
import math
import sys
import random
from pivottablejs import pivot_ui
import sklearn
from sklearn.neighbors import NearestNeighbors

In [2]:
data_users = pd.read_csv('user_cat.csv',index_col=0)
data_users.head()

C:\Users\tusha\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,member_id,category_id,group_id,rating
0,3,2,490552,4.67
1,210,2,490552,4.67
2,357518,2,490552,4.67
3,511008,2,490552,4.67
4,915553,2,490552,4.67


In [3]:
data_users.size

12511788

In [4]:
data_users.shape

(3127947, 4)

In [5]:
tester = data_users.sample(n=1000).copy()
tester.head()

,member_id,category_id,group_id,rating
1685139,179871622,26,1811699,3.67
245475,13286903,34,163773,4.13
831042,185010594,30,368356,4.62
1336282,144858032,34,5763522,4.80
1816640,227147043,28,7009672,4.66


We will be using tester now on

In [6]:
tester.shape

(1000, 4)

In [7]:
tester['values']=1

In [8]:
tester.head()

,member_id,category_id,group_id,rating,values
1685139,179871622,26,1811699,3.67,1
245475,13286903,34,163773,4.13,1
831042,185010594,30,368356,4.62,1
1336282,144858032,34,5763522,4.80,1
1816640,227147043,28,7009672,4.66,1


In [9]:
df_pivot=tester.pivot_table(index=['member_id'], columns=['category_id'],values='values').fillna(0)
df_pivot.head()

category_id,1,2,3,4,5,6,8,9,10,11,...,26,27,28,29,30,31,32,33,34,36
member_id,,,,,,,,,,,,,,,,,,,,,
205871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
361711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
479809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
696461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1075409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df_pivot.shape

(997, 31)

In [11]:
# Pearson Score
# -------------
def pearson_calc(numbers_x, numbers_y):
    if(len(numbers_x)==0):
        mean_x = sum(numbers_x)/0.001
    elif(len(numbers_y)==0):
        mean_y = sum(numbers_y)/0.001
    else:
        mean_x = sum(numbers_x)/len(numbers_x)
        mean_y = sum(numbers_y)/len(numbers_y)
    
    subtracted_mean_x = [i - mean_x for i in numbers_x]
    subtracted_mean_y = [i - mean_y for i in numbers_y]

    x_times_y = [a * b for a, b in list(zip(subtracted_mean_x, subtracted_mean_y))]

    x_squared = [i * i for i in numbers_x]
    y_squared = [i * i for i in numbers_y]
    
    N = sum(x_times_y)
    D = math.sqrt(sum(x_squared) * sum(y_squared))
    if (D !=0):
        return N/D
    else:
        return N/0.001

In [12]:
def match_cal(user1,user2):
    match = 0
    df_u1 = user1.values.tolist()
    df_u2 = user2.values.tolist()
    df_u1 =  [y for x in df_u1 for y in x] 
    df_u2 =  [y for x in df_u2 for y in x] 
    for i in range(0,len(df_u2)):
        if(df_u2[i]==1 and df_u1[i]==1):
            match=match+1
        else:
            match=match+0.001
#     print('match',match)
    pearson_correlation_val = pearson_calc(df_u1,df_u2)
    return match,pearson_correlation_val

In [13]:
def pearson_score(user1,user2):
    '''
    user1 & user2 : user ids of two users between which similarity score is to be calculated.
    '''
    # A list of categories joined by both the users.
    both_match_count, score = match_cal(user1,user2)
    
#     print('both matches :',both_match_count)
    # Returning '0' if no common values.
    if both_match_count == 0 :
        return 0
    return score

In [14]:
score_per_user_match = []
member_vals = []
index_list = []
for index,row in df_pivot.iterrows():
    index_list.append(index)

# print(index_list)

In [15]:
scores = []
members1 = []
members2 = []
result = []
for i in range(0,len(index_list)):
#     print('user 1 --',user1)
    members1.append(index_list[i])
    user1 = df_pivot.iloc[i:i+1]
    for j in range(0,len(index_list)):
        user2 = df_pivot.iloc[j:j+1]
#         print('user 2 --',user2)
        score = pearson_score(user1,user2)
        scores.append(score)
        members2.append(index_list[j])
        list_add = []
        list_add.append(index_list[i])
        list_add.append(index_list[j])
        list_add.append(score)
        result.append(list_add)

In [16]:
checking = pd.DataFrame(result, columns = ['users_A', 'users_B','scores']) 
checking.head()

,users_A,users_B,scores
0,205871,205871,0.967742
1,205871,361711,-0.032258
2,205871,479809,0.967742
3,205871,696461,-0.032258
4,205871,1075409,-0.032258


In [17]:
df_pivot_new=checking.pivot_table(index=['users_A'], columns=['users_B'], values = 'scores').fillna(0)
df_pivot_new.head()

users_B,205871,361711,479809,696461,1075409,1157785,1328135,1777725,2063939,2127276,...,238802156,238819812,239003628,239059981,239313752,239513852,239540835,239930430,240244698,240615881
users_A,,,,,,,,,,,,,,,,,,,,,
205871,0.967742,-0.032258,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,0.967742,...,-0.032258,0.967742,-0.032258,-0.032258,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258
361711,-0.032258,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,...,0.967742,-0.032258,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258
479809,0.967742,-0.032258,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,0.967742,...,-0.032258,0.967742,-0.032258,-0.032258,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258
696461,-0.032258,-0.032258,-0.032258,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,...,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258
1075409,-0.032258,-0.032258,-0.032258,-0.032258,0.967742,0.967742,-0.032258,-0.032258,-0.032258,-0.032258,...,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258


In [18]:
def find_k_neighbours(df,k):
    order = np.argsort(df.values, axis=1)[:, :k]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False).iloc[:k].index, index=['top{}'.format(i) for i in range(1, k+1)]), axis=1)
    return df

In [19]:
neigh_5 = find_k_neighbours(df_pivot_new,5)
neigh_5.head()

,top1,top2,top3,top4,top5
users_A,,,,,
205871,173113422,185017944,152533922,152543402,153454322
361711,184450269,186591421,21593051,21727261,208163926
479809,173113422,185017944,152533922,152543402,153454322
696461,210145815,185049293,226344119,12600769,22402361
1075409,67642342,4248633,220231294,1075409,1157785


In [20]:
from random import sample

In [21]:
user_list = list(neigh_5.index)
user_list = sample(user_list, 5)
print(user_list)

[15413041, 189166959, 237388947, 195401981, 6849809]


In [22]:
members_data_extracted = pd.read_csv('members.csv', encoding = 'Latin-1')
filter_members_user_list = members_data_extracted['member_id'].isin(user_list)
members_data_extracted.where(filter_members_user_list,inplace=True)
members_data_extracted = members_data_extracted.dropna()

In [23]:
members_data_extracted.head()

,member_id,bio,city,country,hometown,joined,lat,link,lon,member_name,state,member_status,visited,group_id
358242,6849809.0,not_found,New York,us,not_found,2008-03-28 23:19:57,40.72,http://www.meetup.com/members/6849809,-73.98,Jeff Rondini,NY,active,2008-08-14 13:14:55,83236.0
1177287,15413041.0,not_found,New York,us,not_found,2014-07-30 14:27:08,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2014-09-16 19:34:36,65784.0
1177288,15413041.0,not_found,New York,us,not_found,2014-07-30 14:23:10,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2016-06-08 23:27:23,1388429.0
1177289,15413041.0,not_found,New York,us,not_found,2011-12-12 17:11:35,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2011-12-12 17:11:35,1455701.0
1177290,15413041.0,not_found,New York,us,not_found,2016-05-31 15:21:08,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2016-06-08 15:09:29,1690232.0


In [24]:
members_data_extracted.shape

(52, 14)

In [25]:
members_data_extracted.to_json(r'selected_members_data.json')

In [26]:
members_data_extracted

,member_id,bio,city,country,hometown,joined,lat,link,lon,member_name,state,member_status,visited,group_id
358242,6849809.0,not_found,New York,us,not_found,2008-03-28 23:19:57,40.72,http://www.meetup.com/members/6849809,-73.98,Jeff Rondini,NY,active,2008-08-14 13:14:55,83236.0
1177287,15413041.0,not_found,New York,us,not_found,2014-07-30 14:27:08,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2014-09-16 19:34:36,65784.0
1177288,15413041.0,not_found,New York,us,not_found,2014-07-30 14:23:10,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2016-06-08 23:27:23,1388429.0
1177289,15413041.0,not_found,New York,us,not_found,2011-12-12 17:11:35,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2011-12-12 17:11:35,1455701.0
1177290,15413041.0,not_found,New York,us,not_found,2016-05-31 15:21:08,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2016-06-08 15:09:29,1690232.0
1177291,15413041.0,R&D at The Nielsen COmpany,New York,us,not_found,2014-07-30 18:56:59,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2014-07-30 18:56:59,1761388.0
1177292,15413041.0,I'm a product lead at the Nielsen Company.,New York,us,not_found,2011-12-12 16:57:51,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2011-12-12 17:06:32,1792777.0
1177293,15413041.0,not_found,New York,us,not_found,2017-01-30 21:17:33,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2017-01-30 21:17:33,13334832.0
1177294,15413041.0,not_found,New York,us,not_found,2017-01-30 21:23:06,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2017-01-30 21:23:06,18198661.0
1177295,15413041.0,not_found,New York,us,not_found,2016-06-02 18:35:56,40.75,http://www.meetup.com/members/15413041,-73.99,Ron Fish,NY,active,2016-06-08 15:09:35,18657546.0


In [27]:
selected_user = user_list[1]

In [56]:
user_val = df_pivot_new.loc[selected_user:selected_user]

In [57]:
user_val

users_B,205871,361711,479809,696461,1075409,1157785,1328135,1777725,2063939,2127276,...,238802156,238819812,239003628,239059981,239313752,239513852,239540835,239930430,240244698,240615881
users_A,,,,,,,,,,,,,,,,,,,,,
189166959,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,...,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258,-0.032258


In [30]:
selected_users_check = []
neigh_5 = neigh_5.loc[selected_user]
for i in neigh_5:
    if(i not in selected_users_check and (i is not None)):
        selected_users_check.append(i)
member_list=[]
for i in range(0,len(selected_users_check)):
    if(selected_user == selected_users_check[i]):
        pass
    else:
        member_list.append(selected_users_check[i])

In [31]:
selected_users_check

[83312702, 175323542, 10242322, 190058200, 189166959]

In [32]:
# This is what we need from the neighbours
member_list

[83312702, 175323542, 10242322, 190058200]

In [33]:
tester.head()

,member_id,category_id,group_id,rating,values
1685139,179871622,26,1811699,3.67,1
245475,13286903,34,163773,4.13,1
831042,185010594,30,368356,4.62,1
1336282,144858032,34,5763522,4.80,1
1816640,227147043,28,7009672,4.66,1


In [34]:
copy_tester = tester.copy()

In [35]:
copy_tester.head()

,member_id,category_id,group_id,rating,values
1685139,179871622,26,1811699,3.67,1
245475,13286903,34,163773,4.13,1
831042,185010594,30,368356,4.62,1
1336282,144858032,34,5763522,4.80,1
1816640,227147043,28,7009672,4.66,1


In [36]:
filter_groups_mem_info = copy_tester['member_id'].isin(member_list)
copy_tester.where(filter_groups_mem_info,inplace=True)
copy_tester = copy_tester.dropna()
copy_tester

,member_id,category_id,group_id,rating,values
2966749,10242322.0,16.0,4459932.0,4.77,1.0
885385,190058200.0,16.0,145951.0,4.72,1.0
2184833,175323542.0,16.0,9441202.0,4.93,1.0
260294,83312702.0,16.0,3324842.0,4.75,1.0


In [37]:
group_list_vals = copy_tester['member_id'].to_list()

In [38]:
print(group_list_vals)

[10242322.0, 190058200.0, 175323542.0, 83312702.0]


In [39]:
main_data_users = data_users.copy()

In [40]:
member_data_all = main_data_users
filter_groups_mem = member_data_all['member_id'].isin(group_list_vals)
member_data_all.where(filter_groups_mem,inplace=True)
member_data_all = member_data_all.dropna()
member_data_all.head()

,member_id,category_id,group_id,rating
31508,175323542.0,23.0,1753252.0,4.85
56626,190058200.0,23.0,1338658.0,4.95
97370,190058200.0,23.0,5616472.0,4.84
260294,83312702.0,16.0,3324842.0,4.75
262844,175323542.0,16.0,3324842.0,4.75


In [41]:
# member_data_all = pd.read_csv('members.csv', encoding = 'Latin-1')
# filter_groups_mem = member_data_all['member_id'].isin(group_list_vals)
# member_data_all.where(filter_groups_mem,inplace=True)
# member_data_all = member_data_all.dropna()

# group_merge_all = pd.read_csv('groups.csv', encoding = 'Latin-1')
# final_merge = pd.merge(member_data_all,group_merge_all,on='group_id')
# final_merge.head()

In [42]:
member_data_all.shape

(56, 4)

In [43]:
getting_groups = member_data_all['group_id'].to_list()

In [44]:
getting_groups = set(getting_groups)
getting_groups = list(getting_groups)
len(getting_groups)

54

In [45]:
random_5_groups = sample(getting_groups, 5)

In [46]:
random_5_groups

[1814389.0, 1240279.0, 18814226.0, 1261788.0, 13757272.0]

In [47]:
selected_group = random_5_groups[1]

In [48]:
selected_group

1240279.0

In [49]:
member_data_all.head()

,member_id,category_id,group_id,rating
31508,175323542.0,23.0,1753252.0,4.85
56626,190058200.0,23.0,1338658.0,4.95
97370,190058200.0,23.0,5616472.0,4.84
260294,83312702.0,16.0,3324842.0,4.75
262844,175323542.0,16.0,3324842.0,4.75


In [50]:
test_df = member_data_all.copy()

In [51]:
test_df.head()

,member_id,category_id,group_id,rating
31508,175323542.0,23.0,1753252.0,4.85
56626,190058200.0,23.0,1338658.0,4.95
97370,190058200.0,23.0,5616472.0,4.84
260294,83312702.0,16.0,3324842.0,4.75
262844,175323542.0,16.0,3324842.0,4.75


In [52]:
def weighted_average_over_scores(score,weights):
    L = len(score)
    weighted_sum = 0.000
    values_weighted_sum = 0.000
    weight_list_gen = []
    for i in range(0,L):
        total_weight = score[i] * weights[i]
#         print('total_weight :',total_weight)
        weight_list_gen.append(total_weight)
    for i in range(L):
#         weighted_sum = weighted_sum + weight_list_random_gen[i - 1]
        weighted_sum = weighted_sum + weight_list_gen[i - 1]
    for i in range(L):
        weight_list_gen.append(score[i - 1] * weight_list_gen[i - 1])
    for i in range(len(weight_list_gen)):
        values_weighted_sum = values_weighted_sum + weight_list_gen[i - 1]
#     if(weighted_sum == 0):
    if(int(weighted_sum) ==0):
#         print("Something is wrong!")
        return values_weighted_sum / 0.001
    else:
        return values_weighted_sum / weighted_sum

In [53]:
def gbellmf(x):
#     print("x :",x)
#     Generalized Bell function fuzzy membership generator
#     Parameters
#     ----------
#     x : 1d array
#         Independent variable.
#     a : float
#         Bell function parameter controlling width. See Note for definition.
#     b : float
#         Bell function parameter controlling center. See Note for definition.
#     c : float
#         Bell function parameter controlling slope. See Note for definition.

    return 1.00 / (1.00 + np.abs((x - 0.001) / 0.001) ** (2 * 0.001))

In [54]:
score_list = []

In [55]:
for i in range(0,len(random_5_groups)):
    # Retrieve data at each iteration
    # Filter it based on the group ID and calcultate the score based on that
    
    data = pd.read_csv('user_cat.csv',index_col=0)
    data = data #.sample(n=10000)
    
    filter_group = data['group_id']==random_5_groups[i]
    data.where(filter_group,inplace=True)
    data = data.dropna()
    
    print(data.shape)
    print(data)
    
    # Creating a pivot table (User-Category Matrix)
    data['values']=1
    df_pivot = data.pivot_table(index=['member_id'], columns=['category_id'],values='values').fillna(0)
    
    score_per_user_match = []
    member_vals = []
    index_list = []
    
    for index,row in df_pivot.iterrows():
        index_list.append(index)
    
    scores = []
    members1 = []
    members2 = []
    result = []
    weights = []
    #     weight_size = weight_df['group_id'].size
    #     print("weight_size :",weight_size)
    weight_val = 0.0001
    for k in range(0,len(index_list)):
        # print('user 1 --',user1)
        members1.append(index_list[k])
#         user1 = df_pivot.iloc[k:k+1]
        user1 = user_val
        for j in range(0,len(index_list)):
            user2 = df_pivot.iloc[j:j+1]
            # print('user 2 --',user2)

            score = pearson_score(user1,user2)
#             print("Score : ",score)
            scores.append(score)
            # normalize = 1//weight_size
            normalize = 0
            weights.append(gbellmf(score))    

            members2.append(index_list[j])

            list_add = []
            list_add.append(index_list[k])
            list_add.append(index_list[j])
            # score_list_old.append(score)
            list_add.append(score)
            result.append(list_add)

            members_one = members1
            members_two = members2
            # Finding the weighted average
            # print("scores :",scores)
            # print("weights :",weights)

    update = weighted_average_over_scores(scores,weights)
    score_list.append(update)

(134, 4)
           member_id  category_id   group_id  rating
2744608    1002722.0         34.0  1814389.0     0.0
2744609    1237529.0         34.0  1814389.0     0.0
2744610    1418715.0         34.0  1814389.0     0.0
2744611    1991845.0         34.0  1814389.0     0.0
2744612    2493813.0         34.0  1814389.0     0.0
2744613    2836491.0         34.0  1814389.0     0.0
2744614    2882221.0         34.0  1814389.0     0.0
2744615    4069890.0         34.0  1814389.0     0.0
2744616    4630727.0         34.0  1814389.0     0.0
2744617    5442884.0         34.0  1814389.0     0.0
2744618    5914257.0         34.0  1814389.0     0.0
2744619    6524747.0         34.0  1814389.0     0.0
2744620    6674490.0         34.0  1814389.0     0.0
2744621    7460133.0         34.0  1814389.0     0.0
2744622    7465845.0         34.0  1814389.0     0.0
2744623    7506178.0         34.0  1814389.0     0.0
2744624    8882945.0         34.0  1814389.0     0.0
2744625    9190491.0         34.0  18

KeyboardInterrupt: 

In [ ]:
score_list

In [ ]:
scores

In [ ]:
weights